Last updated: 2025-Oct-03

Udaya Maurya (udaya_cbscients@yahoo.com, telegram: https://t.me/udy11)

Source: https://github.com/udy11, https://gitlab.com/udy11

## pyIPREQ

This code solves free-boundary MHD Equilibrium problem for axisymmetric Tokamaks for given poloidal coils and limiter configuration. All values are in SI units unless specifically mentioned.

If you used this code for your research, please cite the GIT repository and the following article: [https://doi.org/10.1063/5.0290301](https://doi.org/10.1063/5.0290301). This article is also available on [arXiv](https://arxiv.org/abs/2507.18324).

ALL YOU NEED TO DO:
1. Prepare an input .py file following the provided sample files
2. Use the name of your input file in the second cell (# Load Input File...)
3. Execute all the cells

INPUTS:
- In *Load Input File...* cell, mention the name of your input file as: _%run -i input_file.py_
- For details about each variable refer to sample files

OUTPUTS:
- In the *Main pyIPREQ program...* cell, the psi, J are the main outputs
- In the *Plot J, psi, B_R, B_Z* cell, contour plots are shown
- In the *Compute pressure,...* cell, several relevant physical quantities are printed and plotted

In [ ]:
import sys
import numpy as np
import scipy.special
import scipy.sparse
import scipy.optimize
import scipy.interpolate
import matplotlib.path
import matplotlib.pyplot as plt
import matplotlib.patches
import shapely.geometry
import skimage.measure

In [ ]:
# Load Input File; re-run this cell if you change anything in input file
%run -i input_iter.py

# Setting some common variables
dr, dz = (r1[1] - r1[0], z1[1] - z1[0])    # spacings in R-Z grid
r2, z2 = np.meshgrid(r1, z1, indexing='xy')
rzb = np.array([(r1[i], z1[0]) for i in range(nr)] +    # storing boundary of R-Z grid
               [(r1[0], z1[j]) for j in range(1, nz)] +
               [(r1[nr-1], z1[j]) for j in range(1, nz)] +
               [(r1[i], z1[nz-1]) for i in range(1, nr-1)])
nrzb = rzb.shape[0]    # number of boundary points
ncc = len(ccoils)    # number of poloidal coils
rzlimbnds = ((np.min(rzlims[:,1]),np.max(rzlims[:,1])), (np.min(rzlims[:,0]),np.max(rzlims[:,0])))    # box containing the limiter
mask_lim = matplotlib.path.Path(rzlims).contains_points(np.column_stack((r2.ravel(), z2.ravel()))).reshape(np.shape(r2))    # 2d array same size as r2, acts as mask with value 1 for points inside limiter, 0 for outside
ito_max, iti_max = 64, 100    # maximum iterations in outer and inner loops (increase only if limits exhaust during iterations)

In [ ]:
def binarySearchNN(a, x):
    ''' (array, num) -> int
        Binary Search Nearest Number in a sorted Array of real numbers
    '''
    na = len(a)
    if na < 2:
        return 0
    i0 = 0
    i1 = na - 1
    asc = -1 + 2 * (a[-1] >= a[0])    # 1 if ascending, -1 if descending
    while i1 - i0 > 1:
        im = (i0 + i1) // 2
        dx = asc * (x - a[im])
        if dx > 0:
            i0 = im
        elif dx < 0:
            i1 = im
        else:
            return im
    if asc * (a[i0] + a[i1] - 2*x) >= 0:
        return i0
    else:
        return i1
def GSfdmatfx(mr, mz, r1, dr, dz):
    ''' This function returns LHS matrix for Grad-Shafranov operator finite-difference scheme
        mr and mz are number of elements along r and z coordinates excluding boundaries
        r1 is 1d array for r-axis. dr, dz are widths in r-z mesh
        Test this function for some simple values and see if output is as expected
        Use its output with scipy.sparse.linalg.spsolve() function
    '''
    mm = mz * mr
    dri = 1.0 / dr
    drih = 0.5 * dri
    dr2i = dri * dri
    e3 = 1.0 / (dz * dz)    # diag ± mr values
    e0 = -2.0 * (dr2i + e3)    # diag values
    vals = []
    rows = []
    cols = []
    for j in range(mz):
        for i in range(mr):
            k = i+mr*j
            vals.append(e0)
            rows.append(k)
            cols.append(k)
            if i >= 1:
                vals.append(dr2i + drih / (r1[i+1]))
                rows.append(k)
                cols.append(k-1)
            if i <= mr-2:
                vals.append(dr2i - drih / (r1[i+1]))
                rows.append(k)
                cols.append(k+1)
            if j >= 1:
                vals.append(e3)
                rows.append(k)
                cols.append(k-mr)
            if j <= mz-2:
                vals.append(e3)
                rows.append(k)
                cols.append(k+mr)
    return scipy.sparse.csc_array((vals, (rows, cols)), shape = (mm, mm))
def GSfdrhsfx(rhsGS, nr, nz, r1, dr, dz):
    ''' This function returns RHS array for Grad-Shafranov operator finite-difference scheme
        rhsGS is RHS in 2d and its boundaries are psi values
        nr and nz are number of elements along r and z coordinates
        r1 is 1d array for r-axis. dr, dz are widths in r-z mesh
        Use its output with scipy.sparse.linalg.spsolve() function
    '''
    dri = 1.0 / dr
    drih = 0.5 * dri
    dr2i = dri * dri
    dz2i = 1.0 / (dz*dz)
    GSfdrhs = rhsGS[1:-1,1:-1].flatten()
    # For j,i index of rhsGS, corresponding index in GSfdrhs should be (j-1)*(nr-2)+(i-1)
    # The four entries belong to top row, bottom row, left columns, right column
    # They are identified with j=1, j=nz-2, i=1, i=nr-2, respectively
    for i in range(1, nr-1):
        GSfdrhs[i-1] -= rhsGS[0,i] * dz2i
        GSfdrhs[(nz-3)*(nr-2)+i-1] -= rhsGS[nz-1,i] * dz2i
    for j in range(1, nz-1):
        GSfdrhs[(j-1)*(nr-2)] -= rhsGS[j,0] * (dr2i + drih/r1[1])
        GSfdrhs[(j-1)*(nr-2)+(nr-3)] -= rhsGS[j,nr-1] * (dr2i - drih/r1[nr-2])
    return GSfdrhs
def rhsGS_coils(ccoils, ccurrs):
    ''' RHS of Grad-Shafranov Equation due to coils '''
    rhsGSc = np.zeros((nz, nr))
    for coil, ic in zip(ccoils, ccurrs):
        nrf = max(2, round(coil[2]/drf))
        nzf = max(2, round(coil[3]/dzf))
        rc = np.linspace(coil[0]-0.5*coil[2], coil[0]+0.5*coil[2], nrf)
        zc = np.linspace(coil[1]-0.5*coil[3], coil[1]+0.5*coil[3], nzf)
        for r in rc:
            for z in zc:
                if ic and (r1[0] <= r <= r1[-1]) and (z1[0] <= z <= z1[-1]):
                    i = round(np.floor((r - r1[0]) / dr))
                    j = round(np.floor((z - z1[0]) / dz))
                    ri = i * dr + r1[0]
                    zj = j * dz + z1[0]
                    dr1 = r - ri
                    dz1 = z - zj
                    ucur = mu0 * ic / (dr**2 * dz**2) / (nrf * nzf)
                    rhsGSc[j,i]     -= ucur * ri      * (dr-dr1) * (dz-dz1)
                    rhsGSc[j,i+1]   -= ucur * (ri+dr) *      dr1 * (dz-dz1)
                    rhsGSc[j+1,i]   -= ucur * ri      * (dr-dr1) * dz1
                    rhsGSc[j+1,i+1] -= ucur * (ri+dr) *      dr1 * dz1
    return rhsGSc
def greenGS(R, Z, Rp, Zp):
    kappa2 = 4.0*R*Rp / ((R+Rp)**2 + (Z-Zp)**2)
    return np.sqrt(R*Rp/kappa2) * ((1.0 - 0.5 * kappa2) * scipy.special.ellipk(kappa2) - scipy.special.ellipe(kappa2))
def psib_coils(ncc, nrzb, ccurrs, greenbc):
    psi = np.zeros(nrzb)
    for i in range(ncc):
        if ccurrs[i]:
            for k in range(nrzb):
                psi[k] += ccurrs[i] * greenbc[k, i]
    return psi * 4.0e-7    # 4.0e-7 = mu0 / np.pi
def psib_plasma(r1, z1, nr, nz, drdz, J, rzb, greenbp):
    nrzb = np.shape(rzb)[0]
    psi = np.zeros(nrzb)
    for j in range(nz):
        for i in range(nr):
            if J[j,i]:
                icf = 4.0e-7 * J[j,i] * drdz    # 4.0e-7 = mu0 / np.pi
                for k in range(nrzb):
                    psi[k] += icf * greenbp[k, j, i]
    return psi
def findlims2(rzlims, psi):
    eps_cc = 1.0e-6    # used in deciding if a contour is closed
    eps_psi = 1.0e-6    # used in convergence of boundary psi
    ir1 = np.linspace(0,len(r1)-1,len(r1))    # array of indices for r1 & z1, to be used later
    iz1 = np.linspace(0,len(z1)-1,len(z1))
    limiter_path = matplotlib.path.Path(rzlims)
    limiter_mask = limiter_path.contains_points(np.stack((r2.ravel(), z2.ravel()), axis=1)).reshape(r2.shape)
    psi_lim = np.where(limiter_mask, psi, np.nan)
    psi0 = np.nanmin(psi_lim)
    psi1 = np.nanmax(psi_lim)
    dpsi = 0.01 * (psi1 - psi0)
    for k in range(1, 100):
        contours0 = skimage.measure.find_contours(psi_lim, psi0)
        cls = [len(c) for c in contours0]
        if (len(cls) > 0) and (max(cls) > 3):
            break
        psi0 += dpsi
    else:
        print('ERROR: Search failed for minimum psi that has some good enough number of contour points, this suggests some other problem in psi at this stage.')
        sys.exit()
    for k in range(1, 100):
        contours1 = skimage.measure.find_contours(psi_lim, psi1)
        cls = [len(c) for c in contours1]
        if (len(cls) > 0) and (max(cls) > 3):
            break
        psi1 -= dpsi
    else:
        print('ERROR: Search failed for maximum psi that has some good enough number of contour points, this suggests some other problem in psi at this stage.')
        sys.exit()
    ccs0 = [np.linalg.norm(c[0] - c[-1]) < eps_cc for c in contours0]
    ccs1 = [np.linalg.norm(c[0] - c[-1]) < eps_cc for c in contours1]
    if ccs0.count(True) > 1:
        print(f'ERROR: Multiple closed contours for psi {psi0}, this should ideally not happen.')
        sys.exit()
    if ccs1.count(True) > 1:
        print(f'ERROR: Multiple closed contours for psi {psi1}, this should ideally not happen.')
        sys.exit()
    if (ccs0.count(True) == 0) and (ccs1.count(True) == 0):
        for k in range(51):
            psi0 += dpsi
            contours0 = skimage.measure.find_contours(psi_lim, psi0)
            ccs0 = [np.linalg.norm(c[0] - c[-1]) < eps_cc for c in contours0]
            if ccs0.count(True) > 0:
                break
            psi1 -= dpsi
            contours1 = skimage.measure.find_contours(psi_lim, psi1)
            ccs1 = [np.linalg.norm(c[0] - c[-1]) < eps_cc for c in contours1]
            if ccs1.count(True) > 0:
                break
        else:
            print(f'ERROR: No closed psi contour found.')
            sys.exit()
    if (True in ccs0) and (True in ccs1):   # very rare case where psi0 and psi1 both have closed contours within limiter
        c0 = contours0[ccs0.index(True)]
        cr0 = np.interp(c0[:,1], ir1, r1)
        cz0 = np.interp(c0[:,0], iz1, z1)
        coords0 = np.stack((cr0, cz0), axis=1)
        area0 = shapely.geometry.Polygon(coords0).area
        c1 = contours0[ccs1.index(True)]
        cr1 = np.interp(c1[:,1], ir1, r1)
        cz1 = np.interp(c1[:,0], iz1, z1)
        coords1 = np.stack((cr1, cz1), axis=1)
        area1 = shapely.geometry.Polygon(coords1).area
    if True in ccs1:
        ccs0, ccs1 = ccs1, ccs0
        psi0, psi1 = psi1, psi0
        contours0, contours1 = contours1, contours0
    for k in range(100):
        psim = 0.5 * (psi0 + psi1)
        contours = skimage.measure.find_contours(psi_lim, psim)
        ccs = [np.linalg.norm(c[0] - c[-1]) < eps_cc for c in contours]
        if True in ccs:
            psi0 = psim
            contoursm = contours.copy()
            ccsm = ccs.copy()
        else:
            psi1 = psim
        if np.abs(psi1 - psi0) < eps_psi:
            break
    else:
        print('ERROR: Unable to locate LCFS.')
        sys.exit()
    cr = np.interp(contoursm[ccsm.index(True)][:,1], ir1, r1)
    cz = np.interp(contoursm[ccsm.index(True)][:,0], iz1, z1)
    coords = np.stack((cr, cz), axis=1)
    plasma_path = matplotlib.path.Path(coords)
    plasma_mask = plasma_path.contains_points(np.stack((r2.ravel(), z2.ravel()), axis=1)).reshape(r2.shape)
    return coords, plasma_mask, psi0
def initJ0(r2, z2, dr, dz, nr, nz, rzax, rzlims, Ip):
    nlims = rzlims.shape[0]
    i1 = np.argmin([(rzax[0]-rzlims[i,0])**2 + (rzax[1]-rzlims[i,1])**2 for i in range(nlims)])    # find nearest limiter point to rzax
    i0, i2 = (i1-1)%nlims, (i1+1)%nlims    # neighbor limiter points to i1
    if np.abs(rzlims[i0,0]-rzlims[i1,0]) <= np.finfo(float).eps:
        x0 = rzlims[i1,0]
        y0 = max(min(rzlims[i0,1],rzlims[i1,1]), min(rzax[1], max(rzlims[i0,1],rzlims[i1,1])))    # clamps y0 within limiter positions
    else:
        m0 = (rzlims[i0,1]-rzlims[i1,1])/(rzlims[i0,0]-rzlims[i1,0])
        c0 = -rzlims[i1,0] * m0 + rzlims[i1,1]
        x0 = (rzax[0]+rzax[1]*m0-c0*m0)/(1+m0**2)
        y0 = m0 * x0 + c0
    if np.abs(rzlims[i2,0]-rzlims[i1,0]) <= np.finfo(float).eps:
        x2 = rzlims[i1,0]
        y2 = max(min(rzlims[i2,1],rzlims[i1,1]), min(rzax[1], max(rzlims[i2,1],rzlims[i1,1])))    # clamps y0 within limiter positions
    else:
        m2 = (rzlims[i2,1]-rzlims[i1,1])/(rzlims[i2,0]-rzlims[i1,0])
        c2 = -rzlims[i1,0] * m2 + rzlims[i1,1]
        x2 = (rzax[0]+rzax[1]*m2-c2*m2)/(1+m2**2)
        y2 = m2 * x2 + c2
    D = np.min((np.sqrt((rzax[0]-x0)**2+(rzax[1]-y0)**2), np.sqrt((rzax[0]-x2)**2+(rzax[1]-y2)**2)))
    bt = 1.0 / (1.0 + rzax[0]**2)
    J0 = ((r2-rzax[0])**2 + (z2-rzax[1])**2 < D**2) * (1.0 - ((r2-rzax[0])**2 + (z2-rzax[1])**2) / D**2) * (bt*r2 + (1-bt)/(r2))
    return np.zeros_like(J0), J0 * Ip / (np.sum(J0) * dr * dz)
def rzaxlimdist(rzax, rzlims, th):
    ''' finds distance of rzax to given closed contour
        rzlims (assuming first and last points are same)
        in the given direction th. it'll not work if rzlims
        doesn't have a convex kind of shape '''
    xp, yp = rzax
    for k in range(rzlims.shape[0]-1):
        x0, y0 = rzlims[k]
        x1, y1 = rzlims[k+1]
        uid = ((y0 - y1)*np.cos(th) + (-x0 + x1)*np.sin(th))
        vid = ((y0 - y1)*np.cos(th) + (-x0 + x1)*np.sin(th))
        if uid==0 or vid==0:
            continue
        ui = (xp * (y1 - y0) + x1 * (y0 - yp) + x0 * (yp - y1)) / uid
        vi = ((y0 - yp) * np.cos(th) + (xp - x0) * np.sin(th)) / vid
        if (ui > 0) and ((vi >= 0) and (vi <= 1)):
            return ui
def psi_surfaces(psi, psiaxs, psilim, rzaxs, rzlims, RZlcfs, nrho=101):
    nrho=101
    rhos = np.linspace(0.0, 1.0, nrho)
    fpsin = scipy.interpolate.RectBivariateSpline(z1, r1, (psi - psilim) / (psiaxs - psilim), kx=3, ky=3)
    Rrhos = [np.array([rzaxs[0]])]
    Zrhos = [np.array([rzaxs[1]])]
    nth0, nth1 = 100, 500
    # Interpolate input R,Z of LCFS
    ths1 = np.arctan2(RZlcfs[:-1,1]-rzaxs[1], RZlcfs[:-1,0]-rzaxs[0])    # assuming first and last points are same in RZlcfs
    ths1 = np.where(ths1 < 0.0, ths1 + 2.0 * np.pi, ths1)
    ii = np.argsort(ths1)
    ths1, Rlcfs, Zlcfs = ths1[ii], RZlcfs[:-1,0][ii], RZlcfs[:-1,1][ii]
    ths1 = np.append(ths1, 2.0*np.pi+ths1[0])
    Rlcfs = np.append(Rlcfs, Rlcfs[0])
    Zlcfs = np.append(Zlcfs, Zlcfs[0])
    nth = nth1
    ths = np.linspace(0.0, 2.0*np.pi, nth)
    Rlcfs = scipy.interpolate.CubicSpline(ths1, Rlcfs, bc_type='periodic', extrapolate='periodic')(ths)
    Zlcfs = scipy.interpolate.CubicSpline(ths1, Zlcfs, bc_type='periodic', extrapolate='periodic')(ths)
    ft1lcfs = scipy.interpolate.CubicSpline(ths, np.sqrt((Rlcfs - rzaxs[0])**2 + (Zlcfs - rzaxs[1])**2), bc_type='periodic', extrapolate='periodic')    # distance of magnetic axis to LCFS
    # Compute R, Z on all surfaces inside LCFS
    for k in range(1, nrho-1):
        nth = round(rhos[k] * (nth1 - nth0) + nth0)
        ths = np.linspace(0.0, 2.0*np.pi, nth)
        Rrho = np.empty(nth)
        Zrho = np.empty(nth)
        for m in range(nth):
            fpsint = lambda t: fpsin(rzaxs[1] + t * np.sin(ths[m]), rzaxs[0] + t * np.cos(ths[m])) - (1.0 - rhos[k])
            slv = scipy.optimize.root_scalar(fpsint, bracket=(0, ft1lcfs(ths[m])), method='brentq')
            Rrho[m] = rzaxs[0] + slv.root * np.cos(ths[m])
            Zrho[m] = rzaxs[1] + slv.root * np.sin(ths[m])
        Rrhos.append(Rrho)
        Zrhos.append(Zrho)
    Rrhos.append(Rlcfs)
    Zrhos.append(Zlcfs)
    return rhos, Rrhos, Zrhos
def outvars(psi, psiaxs, psilim, rzaxs, rzlims, RZlcfs, J0fac, fpsi, mask_plasma, Bphi0, nrho=101):
    ''' outputs pressure, F, safety factor, plasma betas, internal inductance, LCFS '''
    rhos, Rrhos, Zrhos = psi_surfaces(psi, psiaxs, psilim, rzaxs, rzlims, RZlcfs, nrho=nrho)
    fpsin = scipy.interpolate.RectBivariateSpline(z1, r1, (psi - psilim) / (psiaxs - psilim), kx=3, ky=3)
    # Pressure p
    pres = scipy.integrate.cumulative_trapezoid(J0fac * pp_fx(1-rhos) / Rmaj, x=rhos*(psilim-psiaxs)+psiaxs, initial=0)
    pres = pres - pres[-1]
    # Current Flux Function F
    F2 = 2.0 * scipy.integrate.cumulative_trapezoid(J0fac * ffp_fx(1-rhos) * Rmaj * mu0, x=rhos*(psilim-psiaxs)+psiaxs, initial=0)
    F = np.sqrt(F2 - F2[-1] + (Rmaj * Bphi0)**2)
    # Bpol,pres on flux surfaces
    dpsidR = fpsi.partial_derivative(0, 1)
    dpsidZ = fpsi.partial_derivative(1, 0)
    Bpols, Bphis, press = [], [], []
    for k in range(nrho):
        nth = len(Rrhos[k])
        Bpol = np.empty(nth)
        Bphi = np.empty(nth)
        for m in range(nth):
            Bpol[m] = float(np.sqrt(dpsidR(Zrhos[k][m], Rrhos[k][m])[0,0]**2 + dpsidZ(Zrhos[k][m], Rrhos[k][m])[0,0]**2) / Rrhos[k][m])
            Bphi[m] = float(F[k] / Rrhos[k][m])
        Bpols.append(Bpol)
        Bphis.append(Bphi)
        press.append(np.ones(nth) * pres[k])
    # Safety Factor q
    Bpol_fx = lambda z, r: float(np.sqrt(dpsidR(z, r)[0,0]**2 + dpsidZ(z, r)[0,0]**2) / r)
    q = np.empty(nrho)
    for k in range(1, nrho):
        dq = F[k] / Bpols[k] / Rrhos[k]**2
        q[k] = scipy.integrate.trapezoid(dq, dx=np.sqrt(np.diff(Rrhos[k])**2+np.diff(Zrhos[k])**2)) * 0.5 / np.pi
    q_fx = scipy.interpolate.CubicSpline(rhos[1:], q[1:], extrapolate=True)
    q[0] = q_fx(0.0)
    # Beta βp, βt, βT, βN
    Rrho1 = np.array([r for Rrh1 in Rrhos for r in Rrh1])
    Zrho1 = np.array([z for Zrh1 in Zrhos for z in Zrh1])
    presRZ = scipy.interpolate.griddata((Zrho1, Rrho1), np.array([p for prs1 in press for p in prs1]), (z2, r2), method='cubic', fill_value=0.0)
    Bpol2RZ = (dpsidR(z1, r1)**2 + dpsidZ(z1, r1)**2) / (r2**2) * mask_plasma    # only in plasma region
    Bphi2RZ = scipy.interpolate.griddata((Zrho1, Rrho1), np.array([Bf**2 for Bphi in Bphis for Bf in Bphi]), (z2, r2), method='cubic', fill_value=0.0)
    presvol = 2.0 * np.pi * scipy.integrate.simpson(scipy.integrate.simpson(r2 * presRZ, x=r1, axis=1), x=z1)
    Bpol2vol = 2.0 * np.pi * scipy.integrate.simpson(scipy.integrate.simpson(r2 * Bpol2RZ, x=r1, axis=1), x=z1)
    Bphi2vol = 2.0 * np.pi * scipy.integrate.simpson(scipy.integrate.simpson(r2 * Bphi2RZ, x=r1, axis=1), x=z1)
    betap = 2.0 * mu0 * presvol / Bpol2vol
    betat = 2.0 * mu0 * presvol / Bphi2vol
    betaT = 1.0 / (1.0/betap + 1.0/betat)
    betaN = 100 * Bphi0 * (np.max(Rrho1) - np.min(Rrho1)) * 0.5 * betaT / (Ip * 1.0e-6)
    # Internal Inductance li
    intInd = 2.0 * Bpol2vol / ((mu0 * Ip)**2 * Rmaj)
    # Minor Radius, Ellipticity, Triangularity
    Rgeo = 0.5 * (np.max(Rrhos[-1]) + np.min(Rrhos[-1]))
    Zgeo = 0.5 * (np.max(Zrhos[-1]) + np.min(Zrhos[-1]))
    minorradius = 0.5 * (np.max(Rrhos[-1]) - np.min(Rrhos[-1]))
    ellipticity = 0.5 * (np.max(Zrhos[-1]) - np.min(Zrhos[-1])) / minorradius
    triangularity_upper = (Rgeo - Rrhos[-1][np.argmax(Zrhos[-1])]) / minorradius
    triangularity_lower = (Rgeo - Rrhos[-1][np.argmin(Zrhos[-1])]) / minorradius
    triangularity = 0.5 * (triangularity_upper + triangularity_lower)
    RZgeoparams = np.array([Rgeo, Zgeo, minorradius, ellipticity, triangularity, triangularity_upper, triangularity_lower])
    return rhos, pres, F, q, q_fx(0.95), np.array([betap, betat, betaT, betaN]), intInd, np.transpose([Rrhos[-1], Zrhos[-1]]), RZgeoparams
def Ifb_vert(C1Z, Ip, rzaxs, fbZ):
    return -C1Z * Ip * (rzaxs[1] - fbZ)
def Ifb_horz(C1R, Ip, rzaxs, fbR):
    return -C1R * Ip * (rzaxs[0] - fbR)
def plot_geometry(axs, ccoils, rzlims, ccurrs):
    for c, ic in zip(ccoils, ccurrs):
        if ic == 0:
            axs.add_patch(matplotlib.patches.Rectangle((c[0]-0.5*c[2], c[1]-0.5*c[3]), c[2], c[3], facecolor=(1,1,1,0), edgecolor='silver', linewidth=1))
        else:
            axs.add_patch(matplotlib.patches.Rectangle((c[0]-0.5*c[2], c[1]-0.5*c[3]), c[2], c[3], facecolor=(1,1,1,0), edgecolor='black', linewidth=1))
    axs.plot(rzlims[:,0], rzlims[:,1], linewidth=1, color='gray')

In [ ]:
# Run this cell only once. Re-run ONLY IF you changed any of these in input file: nr, nz, r1, z1, ccoils, drf, dzf

# Storing Green Function values at boundary points due to plasma
greenbp = np.empty((nrzb, nz, nr))
for k in range(nrzb):
    for j in range(nz):
        for i in range(nr):
            greenbp[k,j,i] = greenGS(rzb[k,0], rzb[k,1], r1[i], z1[j])
print('Computed Green function at boundary points due to all grid points')

# storing Grad-Shafranov finite-difference matrix
GSfdmat = GSfdmatfx(nr-2, nz-2, r1, dr, dz)
print('Computed Grad-Shafranov Finite-Difference Matrix')

# Storing Green Function values at boundary points due to coils
greenbc = np.zeros((nrzb, ncc))
for i in range(ncc):
    nrf = max(2, round(ccoils[i][2]/drf))
    nzf = max(2, round(ccoils[i][3]/dzf))
    rc = np.linspace(ccoils[i][0]-0.5*ccoils[i][2], ccoils[i][0]+0.5*ccoils[i][2], nrf)
    zc = np.linspace(ccoils[i][1]-0.5*ccoils[i][3], ccoils[i][1]+0.5*ccoils[i][3], nzf)
    for k in range(nrzb):
        for r in rc:
            for z in zc:
                greenbc[k,i] += greenGS(rzb[k,0], rzb[k,1], r, z)
        greenbc[k,i] = greenbc[k,i] / (nrf * nzf)
print('Computed Green function at boundary points due to all coils')
del nrf, nzf, rc, zc, r, z

In [ ]:
# Vertical Instability
rhsGSfv = np.zeros((nz, nr))
greenbfv = np.zeros((nrzb, 2))
iifbv = False
if C1Z != 0.0:
    drfbv = 0.1 * (z1[-1] - z1[0])    # coils to be placed 10% outside of total Z-grid height
    RZfbv = np.array([[0.5 * (r1[0] + r1[-1]), z1[0] - 0.2],
                      [0.5 * (r1[0] + r1[-1]), z1[-1] + 0.2]])
    for m in range(2):
        for k in range(nrzb):
            greenbfv[k,m] += greenGS(rzb[k,0], rzb[k,1], RZfbv[m,0], RZfbv[m,1])
        if (r1[0] <= RZfbv[m,0] <= r1[-1]) and (z1[0] <= RZfbv[m,1] <= z1[-1]):
            iifbv = True
            ifb = round(np.floor((RZfbv[m,0] - r1[0]) / dr))
            jfb = round(np.floor((RZfbv[m,1] - z1[0]) / dz))
            ri = ifb * dr + r1[0]
            zj = jfb * dz + z1[0]
            dr1 = RZfbv[m,0] - ri
            dz1 = RZfbv[m,1] - zj
            fcur = mu0 / (dr**2 * dz**2)
            rhsGSfv[jfb,ifb]     -= fcur * ri      * (dr-dr1) * (dz-dz1)
            rhsGSfv[jfb,ifb+1]   -= fcur * (ri+dr) *      dr1 * (dz-dz1)
            rhsGSfv[jfb+1,ifb]   -= fcur * ri      * (dr-dr1) * dz1
            rhsGSfv[jfb+1,ifb+1] -= fcur * (ri+dr) *      dr1 * dz1

# Horizontal Instability
rhsGSfh = np.zeros((nz, nr))
greenbfh = np.zeros((nrzb, 2))
iifbh = False
if C1R != 0.0:
    drfbh = 0.1 * (r1[-1] - r1[0])    # coils to be placed 10% outside of total R-grid width
    RZfbh = np.array([[r1[0] - drfbh, 0.5 * (z1[0] + z1[-1])],
                      [r1[-1] + drfbh, 0.5 * (z1[0] + z1[-1])]])
    for m in range(2):
        for k in range(nrzb):
            greenbfh[k,m] += greenGS(rzb[k,0], rzb[k,1], RZfbh[m,0], RZfbh[m,1])
        if (r1[0] <= RZfbh[m,0] <= r1[-1]) and (z1[0] <= RZfbh[m,1] <= z1[-1]):
            iifbh = True
            ifb = round(np.floor((RZfbh[m,0] - r1[0]) / dr))
            jfb = round(np.floor((RZfbh[m,1] - z1[0]) / dz))
            ri = ifb * dr + r1[0]
            zj = jfb * dz + z1[0]
            dr1 = RZfbh[m,0] - ri
            dz1 = RZfbh[m,1] - zj
            fcur = mu0 / (dr**2 * dz**2)
            rhsGSfh[jfb,ifb]     -= fcur * ri      * (dr-dr1) * (dz-dz1)
            rhsGSfh[jfb,ifb+1]   -= fcur * (ri+dr) *      dr1 * (dz-dz1)
            rhsGSfh[jfb+1,ifb]   -= fcur * ri      * (dr-dr1) * dz1
            rhsGSfh[jfb+1,ifb+1] -= fcur * (ri+dr) *      dr1 * dz1

In [ ]:
# Main pyIPREQ program
# Output variables after running this cell: J0, psi, psin, fpsi, rzaxs, RZlcfs, psiaxs, psilim, J0fac
%run -i input_iter.py
rzaxs = np.array([J0R, J0Z])
psi, J0 = initJ0(r2, z2, dr, dz, nr, nz, rzaxs, rzlims, Ip)
psi_bc = psib_coils(ncc, nrzb, ccurrs, greenbc)
rhsGSc = rhsGS_coils(ccoils, ccurrs)
Ifbv = Ifb_vert(C1Z, Ip, rzaxs, fbZ)
Ifbh = Ifb_horz(C1R, Ip, rzaxs, fbR)
print('io  ii erro      erri      Rax     Zax    J0fac', end='')
if C1Z != 0.0:
    print('     Ifbv', end='')
if C1R != 0.0:
    print('     Ifbh', end='')
print()
for ito in range(ito_max):
    psi_bp = psib_plasma(r1, z1, nr, nz, dr * dz, J0, rzb, greenbp)
    psi_out = psi.copy()
    for iti in range(iti_max):
        psi_in = psi.copy()
        rhsGS = rhsGSc - mu0 * J0 * r2
        if iifbv:
            rhsGS += Ifbv * rhsGSfv
        if iifbh:
            rhsGS += Ifbh * rhsGSfh
        psi_bf = psib_coils(2, nrzb, np.array((-Ifbv, Ifbv)), greenbfv) + psib_coils(2, nrzb, np.array((-Ifbh, Ifbh)), greenbfh)
        rhsGS[0,:] = psi_bc[:nr] + psi_bp[:nr] + psi_bf[:nr]
        rhsGS[1:,0] = psi_bc[nr:nr+nz-1] + psi_bp[nr:nr+nz-1] + psi_bf[nr:nr+nz-1]
        rhsGS[1:,nr-1] = psi_bc[nr+nz-1:nr+2*nz-2] + psi_bp[nr+nz-1:nr+2*nz-2] + psi_bf[nr+nz-1:nr+2*nz-2]
        rhsGS[nz-1,1:nr-1] = psi_bc[nr+2*nz-2:] + psi_bp[nr+2*nz-2:] + psi_bf[nr+2*nz-2:]
        GSfdrhs = GSfdrhsfx(rhsGS, nr, nz, r1, dr, dz)
        psi1 = scipy.sparse.linalg.spsolve(GSfdmat, GSfdrhs)
        #psi = np.empty((nz, nr))    # to recast solution in 2d grid
        psi[0,:] = rhsGS[0,:]
        psi[1:,0] = rhsGS[1:,0]
        psi[1:,nr-1] = rhsGS[1:,nr-1]
        psi[nz-1,1:nr-1] = rhsGS[nz-1,1:nr-1]
        psi[1:-1,1:-1] = psi1.reshape((nz-2,nr-2))
        #psi = 0.5 * psi + 0.5 * psi_in
        fpsi = scipy.interpolate.RectBivariateSpline(z1, r1, psi, kx=3, ky=3)    # can also be done using lambda x: scipy.interpolate.interpn((z1, r1), psi, x)
        solv = scipy.optimize.minimize(lambda x: -fpsi.ev(x[0], x[1]), x0 = (0.0, Rmaj), bounds=rzlimbnds)
        if solv.success:
            rzaxs = np.array([solv.x[1], solv.x[0]])
            psiaxs = -solv.fun
        else:
            print('Maxima of psi could not be found...')
            psi = psi_out.copy()
            continue
        RZlcfs, mask_lcfs, psilim = findlims2(rzlims, psi)
        psin = (psilim - psi) / (psilim - psiaxs)
        psin = psin * (0.0 < psin) * (psin <= 1.0) * mask_lcfs
        J0 = (pp_fx(psin)*r2/Rmaj + ffp_fx(psin)*Rmaj/r2)
        J0fac = Ip / (np.sum(J0) * dr * dz)
        J0 = J0 * J0fac
        Ifbv = Ifb_vert(C1Z, Ip, rzaxs, fbZ)
        Ifbh = Ifb_horz(C1R, Ip, rzaxs, fbR)
        delpsi = np.abs(psi - psi_in)
        aerror = np.abs(np.sum(delpsi) / (psilim - psiaxs) / (nr * nz))
        conv_in = (aerror <= tol_in) and np.all((delpsi) <= tol_in)
        if conv_in or ((ito == 0) and (iti > 0)):
            break
    delpsib = np.concatenate((np.abs(psi[0,:]-psi_out[0,:]),
                              np.abs(psi[1:,0]-psi_out[1:,0]),
                              np.abs(psi[1:,nr-1]-psi_out[1:,nr-1]),
                              np.abs(psi[nz-1,1:nr-1]-psi_out[nz-1,1:nr-1])))
    berror = np.abs(np.sum(delpsib) / (psilim - psiaxs) / (2*(nr+nz-2)))
    tol_in = 0.1 * berror
    print(f'{ito:2d} {iti:3d} {berror:.3e} {aerror:.3e} {rzaxs[0]:.4f} {rzaxs[1]: .4f} {J0fac:.2e}', end='')
    if C1Z != 0.0:
        print(f' {Ifbv: .2e}', end='')
    if C1R != 0.0:
        print(f' {Ifbh: .2e}', end='')
    print()
    conv_out = (berror <= tol_out) and np.all(delpsib <= tol_out)
    if conv_out:
        print(f'Outer-loop iterations finished within tolerance {tol_out:.2e}...')
        break
else:
    print('Maximum outer-loop iterations reached, check convergence before proceeding...')
if solv.success:
    print(f'Magnetic Axis R, Z, psi: {rzaxs[0]:.4e}, {rzaxs[1]:.4e}, {psiaxs:.4e}')
else:
    print('Maxima of psi could not be found...')
del ito, iti, psi_bp, psi_out, psi_in, rhsGS, GSfdrhs, psi1, solv, delpsi, aerror, conv_in, delpsib, berror, conv_out

In [ ]:
# Plot J, psi, B_R, B_Z
def cplotter(fig, ax, var, title='', levels=32, linewidths=1, cmap='brg'):
    plot_geometry(ax, ccoils, rzlims, ccurrs)
    ax.plot(RZlcfs[:,0], RZlcfs[:,1], linewidth=1, color='black')
    plot = ax.contour(r2, z2, var, levels=levels, linewidths=linewidths, cmap=cmap)
    ax.set_xlim([r1[0], r1[-1]])
    ax.set_ylim([z1[0], z1[-1]])
    ax.set_xlabel('R (m)')
    ax.set_title(title)
    ax.set_aspect(1)
    ax.grid()
    ax.set_axisbelow(True)
    fig.colorbar(plot)
fig, axs = plt.subplots(1, 4, figsize=(13, 4))
cplotter(fig, axs[0], J0, title=r'$J\,(A/m^2)$', levels=24, linewidths=1, cmap='brg')
axs[0].set_ylabel('Z (m)')
cplotter(fig, axs[1], psi, title=r'$\psi\,(T\cdot m^2)$', levels=32, linewidths=1, cmap='brg')
cplotter(fig, axs[2], -np.gradient(psi, dz, axis=0)/r2, title=r'$B_R\;(T)$', levels=32, linewidths=1, cmap='brg')
cplotter(fig, axs[3], np.gradient(psi, dr, axis=1)/r2, title=r'$B_Z\;(T)$', levels=32, linewidths=1, cmap='brg')
plt.tight_layout(); plt.show(); plt.close(fig)

In [ ]:
# Compute pressure, F, safety factor, beta, internal inductance
rhos, pres, F, q, q95, betas, intInd, rzlcfs, RZgeoparams = outvars(psi, psiaxs, psilim, rzaxs, rzlims, RZlcfs, J0fac, fpsi, mask_lcfs, Bphi0, nrho=101)
print(f'Rg:  {RZgeoparams[0]: .5f}')
print(f'Zg:  {RZgeoparams[1]: .5f}')
print(f'ag:  {RZgeoparams[2]: .5f}')
print(f'κ:   {RZgeoparams[3]: .5f}')
print(f'δ:   {RZgeoparams[4]: .5f}')
print(f'δu:  {RZgeoparams[5]: .5f}')
print(f'δl:  {RZgeoparams[6]: .5f}')
print(f'q0:  {q[0]: .5f}')
print(f'q95: {q95: .5f}')
print(f'qa:  {q[-1]: .5f}')
print(f'βp:  {betas[0]: .5f}')
print(f'βt:  {betas[1]: .5f}')
print(f'βT:  {betas[2]: .5f}')
print(f'βN:  {betas[3]: .5f} %')
print(f'li:  {intInd: .5f}')
fig, axs = plt.subplots(1, 3, figsize=(12, 3))
axs[0].plot(rhos, q, linewidth=1)
axs[0].set_xlabel(r'$1-\bar\psi$')
axs[0].set_title('Safety Factor')
axs[0].grid()
axs[1].plot(rhos, pres, linewidth=1)
axs[1].set_xlabel(r'$1-\bar\psi$')
axs[1].set_title('Pressure (Pa)')
axs[1].grid()
axs[2].plot(rhos, F, linewidth=1)
axs[2].set_xlabel(r'$1-\bar\psi$')
axs[2].set_title('F (A)')
axs[2].grid()
plt.tight_layout(); plt.show(); plt.close()